### feature합치기
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [38]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [39]:
# index column : date
def find_previous_date(df_inv, df_his, date_current):
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)
        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:
            break
    return date2

In [40]:
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [41]:
def combine_data(df_o, df_inv, df_his):
    investor_rate = pd.DataFrame()
    historical_rate = pd.DataFrame()
    date_weekday = pd.DataFrame()
    for date_current in df_o.index:
        date_previous = find_previous_date(df_inv, df_his, date_current)
        # 거래 전날 요일 구하기
        date_temp = {'date': date_current, 'weekday' : date_previous.weekday()}
        df_temp = pd.DataFrame(date_temp, index=[0]).set_index('date')
        date_weekday = pd.concat([date_weekday, df_temp], axis=0)
        date_previous_1 = find_previous_date(df_inv, df_his, date_previous)
        # 거래 전날, 전전날을 확인하고 변화정도 계산하기(find_ratio)
        df_inv_comp = df_inv.loc[[date_previous_1, date_previous]]
        df_his_comp = df_his.loc[[date_previous_1, date_previous]]
        investor_rate = pd.concat([investor_rate, find_ratio(df_inv_comp, date_current).iloc[[-1]]], axis=0)
        historical_rate = pd.concat([historical_rate, find_ratio(df_his_comp, date_current).iloc[[-1]]], axis=0)
    total = pd.concat([investor_rate, historical_rate, date_weekday, df_o ], axis=1)
    return total

In [42]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

In [43]:
hist_column = [ 'date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']
hist_column_m = [ 'date', 'open', 'high', 'low', 'close', 'vol']
inv_column = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc']

In [44]:
# investor.pkl, historical.pkl. 읽기

# ****************************************************************************************************
## 주요: 오늘날짜로 데이터를 분석하고자 하면 pkl directory 내의 .pkl 데이터가 오늘 날짜로 모두 update시켜 놓아야 함.
##   C:\jupyter연습\factors_affecting_stock_price' 디렉토리에 이쓴
#    'get_company_daily_price_since_last_update_to_pickle.ipynb' 과 (historical.pkl과 investors.pkl update 프로그램)
#     'get_common_data_since_last_update_to_pickle (미국, 한국 주요 지표 등 update용)
# ****************************************************************************************************
directory_for_ml = '../data/data_for_ml/'
pkl_directory = '../data/company_pkl/'
for key, val in code.items():
    pkl_name= '{}{}_4class.pkl'.format(directory_for_ml, val[1])
    globals()['df_{}'.format(val[1])] = pd.read_pickle(pkl_name)
    globals()['df_{}'.format(val[1])]['date'] = pd.to_datetime(globals()['df_{}'.format(val[1])]['date']) #change to timestamp
    globals()['df_{}'.format(val[1])]['date'] = globals()['df_{}'.format(val[1])]['date'].dt.date  # change to datetime
    pkl_name= '{}_historical.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'historical')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'historical')] = globals()['df_{}_{}'.format(val[1], 'historical')][hist_column_m]
    # close_cr 행을 없앰(변동이 없는 경우가 빌생하여 divided zero error 발생)
    globals()['df_{}_{}'.format(val[1], 'historical')]['date'] = globals()['df_{}_{}'.format(val[1], 'historical')]['date'].dt.date # change to datetime
    pkl_name= '{}_investors.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'investors')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'investors')]['date'] = globals()['df_{}_{}'.format(val[1], 'investors')]['date'].dt.date # change to datetime
    
    # get combined data (eg. df_sec_combined, df_ktng_combined...)
    globals()['df_{}_{}'.format(val[1], 'combined')] = combine_data(globals()['df_{}'.format(val[1])].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'investors')].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'historical')].set_index('date')
                            )

In [45]:
# column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
#               'open', 'high', 'low', 'close',  'vol', 'weekday', 'value']

# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',
              'open', 'high', 'low', 'close', 'vol','weekday', 'value' ]

In [46]:
dd = df_ktng_combined[column_selected] # company, common, df_o data combined.
dd

,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc,open,high,low,close,vol,weekday,value
2022-06-20,5.957611,1.382777,0.326582,-5.192308,43.800000,0.244039,-0.226131,-5.833333,-1.414000,-8.555556,-1.363636,-inf,-0.006002,-0.003576,-0.004813,0.0,0.639612,4,2
2022-06-21,0.555802,-0.482667,-1.686069,0.642601,3.437500,-0.736189,-1.850649,-0.827586,-0.874396,-3.441176,1.625000,-0.142857,0.008454,0.001196,-0.007255,-0.010791,-0.423148,0,3
2022-06-22,0.085084,-0.499857,1.037552,0.003643,-0.730382,-0.917094,-0.022901,-1.200000,-6.048077,-1.156627,0.261905,-1.166667,-0.00479,-0.003584,0.00609,0.007273,-0.158932,1,2
2022-06-23,-0.655512,0.301026,-1.034130,-0.965884,-0.839552,3.865979,-0.593750,-0.000000,-0.643810,-0.500000,-0.622642,-13.000000,0.002407,0.003597,-0.006053,-0.01083,0.322923,2,1
2022-06-24,3.798095,-0.383436,-62.020000,54.368794,-17.558140,-0.783898,35.788462,13.500000,-6.994652,1.076923,22.750000,-0.250000,-0.004802,-0.004779,0.001218,0.004866,0.227933,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-21,-3.326858,-0.708149,-1.412313,-1.625824,-1.288462,-0.657534,-0.716346,-1.641026,-26.600000,-1.112903,-1.029412,-0.058824,-0.002291,0.016018,0.008121,0.019518,-0.068007,0,3
2023-02-22,-0.355717,1.649351,-1.334435,-2.419856,36.666667,-0.840000,-9.525424,-0.200000,-0.791667,39.142857,57.000000,-1.500000,0.017222,0.018018,0.018412,0.010135,0.467064,1,1
2023-02-23,-1.094085,-0.680964,0.561779,0.464453,0.264602,-16.500000,-1.063618,-1.300000,4.712500,-0.914591,-1.086207,-0.500000,0.013544,0.0,0.00113,-0.006689,0.090614,2,1
2023-02-24,-20.215569,-0.861716,-4.098101,-1.441451,-0.233730,-0.227957,10.718750,-1.833333,-1.352298,-1.000000,33.200000,0.750000,-0.008909,0.006637,0.004515,0.013468,0.086599,3,3


## 주요 지표 읽기

### Read and Combine data 

In [47]:
# common_pkl = ["bok_rate.pkl", "cpi.pkl", "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
#               "fear_greed.pkl", "fed_rate.pkl", "gold.pkl", "ixic_future.pkl", "kor_10yr_bond.pkl",
#               "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
#               "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
#               "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [48]:
# 매일 개장일자에 따른 변화가 있는 항목 추출; 금리변동과 같이 주기적을 발생하는 항목들은 제외
common_pkl = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
               "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [49]:
end_date = datetime.datetime.today().date()
start_date = '20220620'
date_range_ts = pd.date_range(start=start_date, end=end_date)

In [50]:
pkl_common_directory = '../data/common_pkl/'
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')
# for index_name in major_index:
for index_name in common_pkl:
    key_name = index_name[:-4]
#     globals()['df_{}'.format(key_name)] = pd.read_pickle(pkl_common_directory + index_name)
#     df_base = df_base.merge(globals()['df_{}'.format(key_name)].set_index('date').iloc[:, [-1]], \
#                             how='left', left_index=True, right_index=True)
    df_temp = pd.read_pickle(pkl_common_directory + index_name)
    df_base = df_base.merge(df_temp.set_index('date').iloc[:, [-1]], \
                            how='left', left_index=True, right_index=True)

In [51]:
df_base.tail()

,dji_cr,dji_f_cr,dxy_cr,ixic_f_cr,bond_kor_10_cr,bond_kor_2_cr,kosdaq_cr,kospi_cr,krw_cr,ixic_cr,spx_f_cr,sox_cr,spx_cr,bond_usa_10_cr,bond_usa_2_cr,bond_usa_3m_cr,vix_cr,wti_cr
date,,,,,,,,,,,,,,,,,,
2023-02-24,-1.02%,-1.07%,0.59%,-1.72%,-1.51%,-0.59%,-0.56%,-0.63%,+1.28%,-1.69%,-1.07%,-1.80%,-1.05%,+1.46%,+2.34%,+0.02%,+2.51%,+1.23%
2023-02-25,NaN,NaN,NaN,NaN,-0.40%,-0.79%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-26,NaN,+0.05%,NaN,+0.17%,0.00%,0.00%,NaN,NaN,+0.03%,NaN,+0.09%,NaN,NaN,-0.30%,-0.13%,+0.05%,NaN,+0.43%
2023-02-27,+0.22%,+0.21%,-0.53%,+0.55%,+2.50%,+3.62%,+0.18%,-0.87%,+0.30%,+0.63%,+0.21%,+0.55%,+0.38%,-0.34%,-0.43%,+0.72%,-3.32%,-1.19%
2023-02-28,NaN,+0.14%,0.13%,+0.28%,+2.16%,+1.75%,NaN,+0.87%,-0.18%,NaN,+0.18%,NaN,NaN,+0.10%,+0.13%,-0.21%,0.00%,-0.20%


#### combine with 'df_xxx_combined'

In [52]:
df_sec_sel = df_sec_combined[column_selected]
df_hyunmotor_sel = df_hyunmotor_combined[column_selected]
df_naver_sel = df_naver_combined[column_selected]
df_ktng_sel = df_ktng_combined[column_selected]

In [53]:
df_sec_sel = df_sec_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_hyunmotor_sel = df_hyunmotor_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_naver_sel = df_naver_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_ktng_sel = df_ktng_sel.merge(df_base, how='left', left_index=True, right_index=True)

In [56]:
# 'close_cr' column 추가
pkl_d = '../data/company_pkl/'
for key, val in code.items():
    pkl_name= '{}_historical.pkl'.format(val[1])
#     df = pd.read_pickle(pkl_d + pkl_name).set_index('date')[['close_cr']]
    df = pd.read_pickle(pkl_d + pkl_name).set_index('date')
    df = df.apply(pd.to_numeric)
    min_rate = 0.005 # 수수료등 비용 0.2672%
    df['close_cr_5'] = df['close_cr'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    # 전일대비 +min_rate 이상이면 1, 이하이면  0으로 변환  
    df['r_open_high'] = (df['high'] - df['open'])/df['open'] # 시작가격 이후에 최고가가 min_rate 이상이면 1, 아니면 0
    df['r_open_high_5'] = df['r_open_high'].map(lambda x : 0 if x < min_rate else ( 1 if x >= min_rate  else  "" ))
    
    min_rate = 0.01 # 수수료등 비용 0.2672%
    df['close_cr_1'] = df['close_cr'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    # 전일대비 +min_rate 이상이면 1, 이하이면  0으로 변환  
    df['r_open_high_1'] = df['r_open_high'].map(lambda x : 0 if x < min_rate else ( 1 if x >= min_rate  else  "" ))
    df =  df[['close_cr_5', 'r_open_high_5', 'close_cr_1', 'r_open_high_1']]
#     # # 시작가이후에 최고가가 나오는지 high가 양수면 1, 음수면 0
    globals()['df_{}_sel'.format(val[1])] \
    = globals()['df_{}_sel'.format(val[1])].merge(df, how='left', left_index=True, right_index=True)

In [57]:
# new_columns = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity', 'open',
#   'high', 'low', 'close', 'vol', 'dji_cr', 'ixic_cr', 'spx_cr', 'kosdaq_cr', 'kospi_cr', 'weekday', 'value', 'close_cr']

# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
new_columns = ['retail', 'foreigner', 'institution', 'financial', 'invtrust',
                'pension', 'privequity', 'bank', 'insurance', 'financeetc',
                'corporateetc', 'foreigneretc', 'open', 'high', 'low', 'close', 'vol',
                'dji_cr', 'dji_f_cr', 'ixic_cr', 'ixic_f_cr', 'spx_cr', 'spx_f_cr',
                'bond_kor_10_cr', 'bond_kor_2_cr', 'dxy_cr', 'bond_usa_10_cr','bond_usa_2_cr', 'bond_usa_3m_cr',
                'kosdaq_cr', 'kospi_cr', 'krw_cr',
                'sox_cr', 'vix_cr', 'wti_cr', 
                'weekday', 'value', 'close_cr_5', 'r_open_high_5', 'close_cr_1', 'r_open_high_1']

In [58]:
# column 순서 변경 : weekday, value column을 마지막으로 이동 
df_sec_sel =  df_sec_sel[new_columns]
df_hyunmotor_sel =  df_hyunmotor_sel[new_columns]
df_naver_sel =  df_naver_sel[new_columns]
df_ktng_sel =  df_ktng_sel[new_columns]

In [59]:
'''
# drop inf, -inf : first replace inf to NaN [ bank to foreigneretc]
df_sec_sel.replace([np.inf, -np.inf], np.nan, inplace=True)
df_hyunmotor_sel.replace([np.inf, -np.inf], np.nan, inplace=True)
df_naver_sel.replace([np.inf, -np.inf], np.nan, inplace=True)
df_ktng_sel.replace([np.inf, -np.inf], np.nan, inplace=True)
'''

'\n# drop inf, -inf : first replace inf to NaN [ bank to foreigneretc]\ndf_sec_sel.replace([np.inf, -np.inf], np.nan, inplace=True)\ndf_hyunmotor_sel.replace([np.inf, -np.inf], np.nan, inplace=True)\ndf_naver_sel.replace([np.inf, -np.inf], np.nan, inplace=True)\ndf_ktng_sel.replace([np.inf, -np.inf], np.nan, inplace=True)\n'

In [60]:
# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고
# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지
df_sec_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)
df_hyunmotor_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)
df_naver_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)
df_ktng_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)

In [61]:
# delete rows which include NaN : dji, spx, nasdaq 지수가 nan인 rows 제거
df_sec_sel.dropna(inplace=True)
df_hyunmotor_sel.dropna(inplace=True)
df_naver_sel.dropna(inplace=True)
df_ktng_sel.dropna(inplace=True)

In [62]:
# string column을 numeric으로 전환
def string_to_num(df):
    df.replace('%', '', regex=True, inplace=True)
    return df.apply(pd.to_numeric)

df_sec_sel = string_to_num(df_sec_sel)
df_hyunmotor_sel = string_to_num(df_hyunmotor_sel)
df_naver_sel = string_to_num(df_naver_sel)
df_ktng_sel = string_to_num(df_ktng_sel)

In [63]:
# pickle로 데이터 저장
directory_for_ml = '../data/data_for_ml/'
df_sec_sel.to_pickle(directory_for_ml + 'df_sec_sel.pkl')
df_hyunmotor_sel.to_pickle(directory_for_ml + 'df_hyunmotor_sel.pkl')
df_naver_sel.to_pickle(directory_for_ml + 'df_naver_sel.pkl')
df_ktng_sel.to_pickle(directory_for_ml + 'df_ktng_sel.pkl')

df_sec_sel.to_csv(directory_for_ml + 'df_sec_sel.csv')
df_hyunmotor_sel.to_csv(directory_for_ml + 'df_hyunmotor_sel.csv')
df_naver_sel.to_csv(directory_for_ml + 'df_naver_sel.csv')
df_ktng_sel.to_csv(directory_for_ml + 'df_ktng_sel.csv')

### feature 이름 설명
#### feature에 사용된 column name 설명

In [64]:
df_feature_name_abbr = pd.read_csv('../data/feature_name_abbr.csv')

In [26]:
standforK = lambda x : df_feature_name_abbr['Korean'][df_feature_name_abbr['abbr'] == x].values[0]
standforE = lambda x : df_feature_name_abbr['English'][df_feature_name_abbr['abbr'] == x].values[0]
standforexp = lambda x : df_feature_name_abbr['exp1'][df_feature_name_abbr['abbr'] == x].values[0]

In [27]:
# financial이 무엇인지 알기 위한 명령
c_name = 'dxy_cr'
standforK(c_name), standforE(c_name), standforexp(c_name)

('미국달러지수 변동률', 'Dollar_index_change_rate', nan)

In [28]:
c_name = 'cpi'
standforK(c_name), standforE(c_name), standforexp(c_name)

('소비자물가지수', 'Consumer Price Index', nan)

In [29]:
c_name = 'sox'
standforK(c_name), standforE(c_name), standforexp(c_name)

('필라델피아 반도체지수', 'SOX_INDEX', nan)

### 설명에서 feature name 찾기

In [30]:
find_col_nameK = lambda x : df_feature_name_abbr[['abbr', 'Korean']][df_feature_name_abbr['Korean'].str.contains(x)]
find_col_nameE = lambda x : df_feature_name_abbr[['abbr', 'English']][df_feature_name_abbr['English'].str.contains(x, na=False, case=False)]

In [31]:
e_word = 'INDEX' # English column에서 찾기
find_col_nameE(e_word)

,abbr,English
68,dxy,Dollar_index
69,dxy_cr,Dollar_index_change_rate
84,f_g_index,FEAR_GREED_INDEX
85,sox,SOX_INDEX
86,sox_cr,SOX_INDEX_change_rate
87,cpi,Consumer Price Index
88,cpi_anticipated,anticipated consumer price index
89,cpi_previous,previous consumer price index


In [32]:
k_word = '반도체' # Korean column에서 찾기
find_col_nameK(k_word)

,abbr,Korean
85,sox,필라델피아 반도체지수
86,sox_cr,필라델피아 반도체지수 변동률


#### pd.merge 형식 참조
df_base = df_base.merge(djis, how='left', left_index=True, right_index=True)

In [33]:
bokd = '../data/common_pkl/bok_rate.pkl'
cpid = '../data/common_pkl/cpi.pkl'
djid = '../data/common_pkl/dji.pkl'
bok = pd.read_pickle(bokd)
cpi = pd.read_pickle(cpid)
dji = pd.read_pickle(djid)
boks = pd.read_pickle(bokd).set_index('date')
cpis = pd.read_pickle(cpid).set_index('date')
djis = pd.read_pickle(djid).set_index('date')

In [34]:
end_date = datetime.datetime.today().date()
start_date = '20220620'
date_range_ts = pd.date_range(start=start_date, end=end_date)

In [35]:
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')

In [36]:
df_base = df_base.merge(cpis, how='left', left_index=True, right_index=True)

In [37]:
df_base = df_base.merge(djis, how='left', left_index=True, right_index=True)